# Tutorial

This is a tutorial for MB-MVPA using task-fMRI data of Mixed-gamble task by Tom et al., 2007. 



### Import the MB-MVPA libarary.

Other libraries(nilean, keras, etc..) dosen't need to be imported.<br>
Because mb-mvpa has wrapping the libararies.<br>
You don't necessarily have to know fMRI libraries like nilearn and machine learning libraries like tensorflow.<br>
<b>MB-MVPA is all you need.</b>

Most of mb-mvpa are wrapping nilearn, tensorflow, Keras and etc., so warning can occur from that libraries.<br>
This page does not print warning because most of them are can be ignored.<br>
You don't need to remove the warning when you are actually using it.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from mbmvpa.preprocessing.preprocess import DataPreprocessor

TODO: add original data download link

Data download from AWS S3, ~ <b>1GB</b> (would be under the "Mixed-gamble_task/example_data/").<br>

We provide a small subset (2 subjects) of original Tom's dataset (16 subjects). The fMRI images in the example is preprocessed by conventional fMRI preprocessing pipeline by using 
[*fmriprep*](https://fmriprep.org/en/stable/) v.20.1.0. Please refer to the [original](https://openneuro.org/datasets/ds000005/versions/00001) for more information.

In [3]:
#root = load_example_data("tom")
root = "/data2/project_modelbasedMVPA/ds000005"

### Preprocessing fMRI images and behavioral data

MB-MVPA requires primariliy preprocessed task-fMRI experiments data fromatted in conventional [BIDS format](https://bids-specification.readthedocs.io/en/stable/) 

It expects the following organized files. All the naming conventions used here conform with outputs from *fmriprep* v.20.1.0. by Poldrack lab.

The fMRI images are usually located here<br>
<i>{BIDS_ROOT}/derivatives/fmriprep/subject/session/run/func/*nii.gz</i><br>
And the behavior data are located here<br>
<i>{BIDS_ROOT}/subject/session/run/func/*.tsv</i>

In [5]:
dm_model = 'ra_prospect'

def example_adjust(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

def example_filter(row):
    # include all trial data
    return True

def example_latent(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row


preprocessor = DataPreprocessor(bids_layout=root,
                               adjust_function=example_adjust,
                               filter_function=example_filter,
                               latent_function=example_latent,
                               dm_model=dm_model)


In [6]:
preprocessor.summary()

[  fMRIPrep  ] BIDS Layout: .../ds000005/derivatives/fmriprep | Subjects: 16 | Sessions: 0 | Runs: 48
[  MB-MVPA   ] BIDS Layout: ...PA/ds000005/derivatives/mbmvpa | Subjects: 0 | Sessions: 0 | Runs: 0


In [ ]:
preprocessor.preprocess(overwrite=True)

In [ ]:
preprocessor.summary()

### Load data and shape check

Through the above process, <b><i>subject, session and run-wise fMRI images and modulation values are obtained</i></b><br>
<b><i>Both X(fMRI), y(modulation)'s shape[0] must equal.</i></b><br>
The <b><i>prepare_dataset</i></b> function operate shape check and time and voxel masking.

``` python
def prepare_dataset(
    root, # (str or pathlib.Path): Path where created data (X, y, time_mask, voxel_mask) is stored.
    time_masking=True, # (bool): Whether to do time masking or not.
    voxel_masking=True # (bool): Whether to do voxel masking or not.
)
```

In [ ]:
X, y, voxel_mask = prepare_dataset(data_root)

<b><i>X (numpy.ndarray)</i></b>: X, which is adjusted dimension and masked time points for training with shape: data # x voxel #.<br>
<b><i>y (numpy.ndarray)</i></b>: y, which is adjusted dimension and masked time points for training with shape: data #.<br>
<b><i>voxel_mask (nibabel.nifti1.Nifti1Image)</i></b>: Voxel mask file for get result brain map.

### Fitting MVPA models & Results

In [ ]:
from mbmvpa.utils.coef2map import get_map
from time import perf_counter

In [ ]:
from mbmvpa.models.regressor import penalized_linear_regression

s = perf_counter()
coefs = penalized_linear_regression(X, y,
                                    layout,
                                    lambda_param=2.0,
                                    N=10,
                                    verbose=1)
result = get_map(coefs, voxel_mask, task_name="tom2007_penalized_linear", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

In [ ]:
from mbmvpa.models.regressor import mlp_regression

s = perf_counter()
coefs = mlp_regression(X, y,
                       layout,
                       layer_dims=[1024, 1024],
                       activation="linear",
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=3,
                       verbose=1)
result = get_map(coefs, voxel_mask, task_name="tom2007_mlp", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

In [ ]:
from mbmvpa.models.regressor import elasticnet

s = perf_counter()
coefs = elasticnet(X, y,
                   layout,
                   n_jobs=16,
                   verbose=1,
                   max_lambda=1,
                   n_samples=5000)
result = get_map(coefs, voxel_mask, task_name="tom2007_elasticnet", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")